# The Qmod Workshop - Introduction

The Classiq platform features a high-level quantum modeling language called Qmod. Qmod is compiled into concrete gate-level implementation using a powerful synthesis engine that optimizes and adapts the implementation to different target hardware/simulation environments.

In this workshop, we will learn how to write quantum models using Qmod. We will be using the Python embedding of Qmod, available as part of the Classiq Python SDK. We will learn basic concepts in the Qmod language, such as functions, operators, quantum variables, and quantum types. We will develop useful building blocks and small algorithms.

The [QMOD language reference](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/) covers these concepts more systematically and includes more examples.

This workshop consists of step-by-step exercises. It is structured as follows:

- Part 1: Language Fundamentals - Exercises 1-5
- Part 2: Higher-Level Concepts - Exercises 6-10
- Part 3: Execution Flows - Exercises 11, 12

The introduction and Part 1 are included in this notebook. Part 2 and 3 are each in its own separate notebook. For each exercise you will find the solution to the exercises at the bottom of the same notebook.

### Preparations

Make sure you have a Python version of 3.8 through 3.11 installed. Unfortunately, Classiq is not yet supported with Python 3.12.

Install Classiq’s Python SDK by following the instructions on this page: [Getting Started - Classiq](https://docs.classiq.io/latest/getting-started/).

### Python Qmod Exercises - General Instructions

In order to synthesize and execute your Qmod code, you should:
1. Make sure you define a `main` function that calls functions you create.
2. Use `create_model` by running `qmod = create_model(main)` to construct a representation of your model.
3. You can synthesize the model (using `qprog = synthesize(qmod)`) to obtain an implementation - a quantum program.
4. You can then visualize the quantum program (`show(qprog)`) or execute it (using `execute(qprog)`. See: [Execution - Classiq](https://docs.classiq.io/latest/user-guide/platform/executor/#full-example)). You can also execute it with the IDE after visualizing the circuit.


### Exercise 0: From Model to Execution

The following model defines a function that applies X and H gates on a single qubit, and subsequently calls it:

In [1]:
from classiq import *


# Define a quantum function using the @qfunc decorator
@qfunc
def foo(q: QBit) -> None:
    X(target=q)
    H(target=q)


# Define a main function
@qfunc
def main(res: Output[QBit]) -> None:
    allocate(1, res)
    foo(q=res)

Create a model from it, and synthesize, visualize, and execute it.

Use the General Instructions above to do so.


In [2]:
from classiq import *

# Your code here:
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)

Opening: https://platform.classiq.io/circuit/0217ea7a-3f50-4498-b139-ab68ff8a1101?version=0.41.2


In Qmod `QBit` is the simplest quantum type, and in this example, `q` is a quantum variable of type `QBit`. Quantum variables abstract away the mapping of quantum objects to qubits in the actual circuit.

See also [Quantum Variables](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/quantum-variables/).

We will discuss other quantum types during the workshop.


# The Qmod Workshop - Part 1: Language Fundamentals

Follow exercises 1 through 5 for the first session of the workshop.

## Exercise 1 - Bell Pair

Create a function that takes two single-qubit (`QBit`) quantum arguments and prepares the bell state on them ([Bell state](https://en.wikipedia.org/wiki/Bell_state)) by applying `H` on one variable and then using it as the control of a `CX` function with the second variable as the target.
Create a main function that uses this function and has two single-qubit outputs, initialize them to the |0> state (using the `allocate` function), and apply your function to them.

See also [Functions](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/functions#syntax)


In [15]:
from classiq import *

# Your code here:
@qfunc
def my_function(x:QBit, y:QBit):
    H(x)
    CX(x, y)

@qfunc
def main(x: Output[QBit], y: Output[QBit]):
    allocate(1,x)
    allocate(1,y)
    my_function(x,y)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/f0770ecf-0463-46f4-b97b-3c7326ff1aad?version=0.41.2


Use qubit array subscript (the syntax - _variable_ **[** _index-expression_ **]**) to change the function from subsection 1 to receive a single quantum variable, a qubit array (`QArray`) of size 2.
Change your main function to declare a single output (also an array of size 2).


In [8]:
from classiq import *

# Your code here:
@qfunc
def my_function(reg: QArray[QBit]):
    H(reg[0])
    CX(reg[0], reg[1])

@qfunc
def main(reg: Output[QArray]):
    allocate(2,reg)
    my_function(reg)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/eff4d409-98f8-4608-b2c5-1e84d91d2427?version=0.41.2


## Exercise 2 - Repeat

Use the built-in `repeat` operator to create your own Hadamard transform function (call it `my_hadamard_transform`). The Hadamard transform function is a function that takes as argument a qubit array of an unspecified size and applies `H` to each of its qubit.

See also [Classical repeat](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/classical-control-flow/#classical-repeat).

Set your main function to have a quantum array output of unspecified size, allocate 10 qubits, and then apply your Hadamard transform function.


In [17]:
from classiq import *

# Your code here:
@qfunc
def my_hadamard_transform(reg: QArray[QBit]):
    repeat(reg.len, lambda i: H(reg[i]))

@qfunc
def main(reg: Output[QArray]):
    allocate(10, reg)
    my_hadamard_transform(reg)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/00c3f76a-8570-44d9-ac1d-e8d745c0da98?version=0.41.2


### Note: Quantum Variable Capture
The `repeat` operator invokes a statement block multiple times. The statement block is specified using a Python callable, typically a lambda expression. Inside the block you can refer to variables declared in the outer function scope.
This concept is called `quantum variable capture`, equivalent to [capture](https://en.wikipedia.org/wiki/Closure_(computer_programming)) in classical languages.

See also [Capturing context variables and parameters](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/operators/#capturing-context-variables-and-parameters).

### Exercise 3 - Power
Raising a quantum operation to a power appears in many known algorithms, for examples, in Grover search and Quantum Phase Estimation.
For most operations, it simply means repeating the same circuit multiple times.

Sometimes, however, power can be simplified, thus saving computational resources.
The most trivial example is a quantum operation expressed as a single explicit unitary matrix (i.e., all n*n matrix terms are given) -  raising the operation can be done by raising the matrix to that power via classical programming.

See also [Power operator](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/quantum-operators/#syntax).

Use the following code to generate a 2-qubit (real) unitary matrix:

In [24]:
from typing import List

import numpy as np

from classiq import *

rng = np.random.default_rng(seed=0)
random_matrix = rng.random((4, 4))
qr_unitary, _ = np.linalg.qr(random_matrix)

unitary_matrix = QConstant("unitary_matrix", List[List[float]], qr_unitary.tolist())

In order to reuse some classical value we can define a `QConstant` to store that value.

1. Create a model that applies `unitary_matrix` on a 2 qubit variable.
2. Create another model that applies `unitary_matrix` raised to power 3 on a 2 qubit variable.
3. Compare the gate count via the Classiq’s IDE in both cases.

Note - the signature of function `unitary` is:

In [25]:
def unitary(
    elements: CArray[CArray[CReal]],
    target: QArray[QBit],
) -> None:
    pass

In [37]:
from classiq import *

# Your code here:
@qfunc
def unitary1(elements: QArray[QBit]):
    apply_to_all(X, elements)

@qfunc
def main(reg: Output[QArray]):
    allocate(2,reg)
    unitary1(reg)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

## Has a gate count of 2

Opening: https://platform.classiq.io/circuit/3b242d63-67db-4482-aec6-a3761f48a70f?version=0.41.2


In [38]:
@qfunc
def main(reg: Output[QArray]):
    allocate(2,reg)
    unitary1(reg)
    unitary1(reg)
    unitary1(reg)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

## Has a gate count of 6 

Opening: https://platform.classiq.io/circuit/03db2ec5-3f57-48cc-9d61-06184d25a9c8?version=0.41.2


## Exercise 4 - User-defined Operators
Create a function that applies a given single-qubit operation to all qubits in its quantum argument (Call your function `my_apply_to_all`). Such a function is also called an operator, i.e. a function that one of its arguments is another function (its operand).

See also [Operators](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/operators/).

Follow these guidelines:
1. Your function should declare a quantum argument of type qubit array. It should also declare an argument of a function type with a single qubit argument.
2. The body should apply the operand to all qubits in the argument.

When you're done, re-implement `my_hadamard_transform` from exercise 2 using this function instead of `repeat`.
Use the same main function from exercise 2.

In [58]:
from classiq import *

# Your code here:
@qfunc
def my_apply_to_all(my_operand: QCallable[QBit], q: QArray[QBit]):
    #apply_to_all(H, q)
    repeat(q.len, lambda i: H(q[i]))

def my_operand(q: QBit):
    H(q)
    
@qfunc
def main(reg: Output[QArray]):
    allocate(10, reg)
    my_apply_to_all(my_operand, reg)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/d7e0654e-273e-4193-ac8a-f274b276e360?version=0.41.2


# Exercise 5 -  Quantum Conditionals

### Exercise 5a - Control Operator
Use the built-in `control` operator to create a function that receives two single qubit variables and uses one of the variables to control an RY gate with a `pi/2` angle acting on the other variable (without using the `CRY` function).

See also [Quantum operators](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/quantum-operators/#syntax).


In [64]:
from classiq import *
from math import pi
# Your code here:
@qfunc
def my(my_operand: QCallable[CReal, QBit], target: QBit):
    H(target)
    my_operand(pi / 2, target)
    
def my_operand(angle: CReal, target: QBit) -> None:
    RY(angle, target)

@qfunc
def main(r:Output[QBit]) -> None:
    #q = QBit("q")
    allocate(1,r)
    my(my_operand, r)
    
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/78ab17bf-8e0f-4a02-b280-b6f237c1cbaf?version=0.41.2


### Exercise 5b - Control ("Quantum If")
The `control` operator is the conditional application of some operation, with the condition being that all control qubits are in the state |1>. This notion is generalized in QMOD to other control states, where the condition is specified as a comparison between a quantum numeric variable and a numeric value, similar to a classical `if` statement. Quantum numeric variables are declared with class `QNum`.

See also [Numeric types](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/quantum-types/#syntax).

In QMOD this generalization is available as a native statement - control.

See also [control](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/quantum-operators/).

1. Declare a `QNum` output argument using `Output[QNum]` and name it `x`.
2. Use the `prepare_int` function to initialize it to `9`. Note that you don't need to specify the `QNum` attributes - size, sign, and fraction digits, as they are inferred at the point of initialization.
3. Execute the circuit and observe the results.
4. Declare another output argument of type `QBit` and perform a `control` such that under the condition that `x` is 9, the qubit is flipped. Execute the circuit and observe the results. Repeat for a different condition.

In [79]:
from classiq import *

# Your code here:
@qfunc
def cont(target: QBit):
    #n = QNum('n')
    X(target)
    
@qfunc
def main(target: Output[QArray[QBit]], x: Output[QBit]):
    #n = QNum('n')
    allocate(1,x)
    prepare_int(9,target)
    control(ctrl =target, operand=lambda: X(x))

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

# The extra qubit X still remains a zero

Opening: https://platform.classiq.io/circuit/36761b29-e064-4ed2-a2e4-8b178d5b9f2c?version=0.41.2
